## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# nvda 134.97 1000

In [26]:
symbol = 'odfl'.upper()

# 5 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '5m', skip_dl=False) # skip_dl=True on redo's
print(f'{symbol} 5m Interval Timestamp: {time_stamp} ')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 15 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '15m', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 15m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 30 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '30m', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 30m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Hour Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1h', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1h Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Day Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1d', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1d Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Week Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1wk', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1wk Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

ODFL 5m Interval Timestamp: 2024-10-15 11:50:51 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.475,0.918999,0.842367,0.502785
prob_up,0.0,0.031801,0.035726,0.176614
prob_static,0.625,0.942142,0.887405,0.644846
prob_down,0.375,0.026057,0.07687,0.17854
precision,0.672897,0.656388,0.665158,0.557692
recall,0.634361,0.656388,0.647577,0.638767
f1,0.653061,0.656388,0.65625,0.595483
support,"[230.0, 227.0, 228.0]","[230.0, 227.0, 228.0]","[230.0, 227.0, 228.0]","[230.0, 227.0, 228.0]"



ODFL 15m Interval Timestamp: 2024-10-15 11:50:56


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,down,down,static
kelly_1:2.5,0.3,0.299814,0.388686,0.157434
prob_up,0.5,0.052408,0.203117,0.264328
prob_static,0.125,0.447725,0.233536,0.398167
prob_down,0.375,0.499867,0.563347,0.337505
precision,0.78481,0.727273,0.684211,0.634615
recall,0.712644,0.744186,0.755814,0.776471
f1,0.746988,0.735632,0.718232,0.698413
support,"[87.0, 85.0, 86.0]","[87.0, 85.0, 86.0]","[87.0, 85.0, 86.0]","[87.0, 85.0, 86.0]"



ODFL 30m Interval Timestamp: 2024-10-15 11:50:58


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.475,0.845423,0.876126,0.687148
prob_up,0.375,0.072674,0.046913,0.131971
prob_static,0.625,0.889588,0.911519,0.776535
prob_down,0.0,0.037738,0.041568,0.091495
precision,0.74505,0.741602,0.720874,0.579176
recall,0.813514,0.775676,0.802703,0.721622
f1,0.777778,0.758256,0.759591,0.642599
support,"[368.0, 370.0, 369.0]","[368.0, 370.0, 369.0]","[368.0, 370.0, 369.0]","[368.0, 370.0, 369.0]"



ODFL 1d Interval Timestamp: 2024-10-15 11:51:06


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.475,0.330236,0.536598,0.37883
prob_up,0.625,0.521597,0.668999,0.556307
prob_static,0.125,0.474715,0.319885,0.421947
prob_down,0.25,0.003688,0.011117,0.021746
precision,0.785124,0.762846,0.769565,0.647826
recall,0.769231,0.781377,0.716599,0.603239
f1,0.777096,0.772,0.742138,0.624738
support,"[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]"



ODFL 1wk Interval Timestamp: 2024-10-15 11:51:11


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.125,0.983154,0.964184,0.701694
prob_up,0.375,0.987967,0.974417,0.786924
prob_static,0.25,0.009573,0.018474,0.165882
prob_down,0.375,0.00246,0.007109,0.047194
precision,0.75,0.765957,0.755102,0.619048
recall,0.8125,0.75,0.770833,0.541667
f1,0.78,0.757895,0.762887,0.577778
support,"[48.0, 50.0, 48.0]","[48.0, 50.0, 48.0]","[48.0, 50.0, 48.0]","[48.0, 50.0, 48.0]"


In [ ]:
## Sanity Checks

In [ ]:
df = f.load('NVDA','1d')
df.tail(3)

In [ ]:
today, start, df = f.download('NVDA','5m')
df_saved = f.load('NVDA','5m')
display(df.tail(3))
display(df_saved.tail(3))

In [ ]:
str(today)

In [ ]:
str(start)

### Check Summary Output

In [ ]:
symbol='NVDA'
interval='5m'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')